In [ ]:
from motor.motor_asyncio import AsyncIOMotorClient

# DB 연결 클라이언트
# 인자로는 데이터베이스의 연결 문자열
db_client = AsyncIOMotorClient("")

# myGPT 데이터 베이스를 쓰도록 설정
db = db_client['myGPT']


# 데이터 넣기
# 지정힌DB.테이블.insert_one( {JSON 데이터} )
# await db.messages.insert_one({"channel_id": "asdfasdf", "content": "Data_02" })



InsertOneResult(ObjectId('6864ce892d309a62265609bd'), acknowledged=True)

In [ ]:

# 데이터 조회
# 조회는 기본적으로 _id 를 기준으로 오래된 것부터 나옴
await db.messages.find({"channel_id": "asdfasdf"}).to_list()



[{'_id': ObjectId('6864ce572d309a62265609ba'),
  'channel_id': 'asdfasdf',
  'content': 'Data_01'},
 {'_id': ObjectId('6864ce882d309a62265609bc'),
  'channel_id': 'asdfasdf',
  'content': 'Data_02'},
 {'_id': ObjectId('6864ce892d309a62265609bd'),
  'channel_id': 'asdfasdf',
  'content': 'Data_03'}]